In [14]:
import horton
import numpy as np
import pandas as pd
import sys

def get_hf_density(baseline, target, mixing, basisset, label):
    read = horton.io.xyz.load_xyz('benzene.xyz')
    
    target = np.array(target)
    baseline = np.array(baseline)
    dZ = target - baseline
    
    horton.log.set_level(0)
    mol = horton.IOData()
    mol.coordinates = read['coordinates']
    mol.numbers = baseline
    mol.pseudo_numbers = baseline + dZ * mixing
    
    # get grid
    grid = horton.BeckeMolGrid(mol.coordinates, np.array([6, 6, 6, 6, 6, 6, 1, 1, 1, 1, 1, 1]), np.array([6, 6, 6, 6, 6, 6, 1, 1, 1, 1, 1, 1]), 'insane', mode='keep', random_rotate=False)
    
    # build basis set
    basisset = basisset
    obasis = horton.get_gobasis(mol.coordinates[:6], baseline[:6].astype(int), basisset)
    for idx, dval in enumerate(dZ):
        if dval == 0:
            continue
        obasis2 = horton.get_gobasis(mol.coordinates[idx:idx+1], target[idx:idx+1].astype(int), basisset)
        obasis = horton.GOBasis.concatenate(obasis, obasis2)
    
    # start calculation
    olp = obasis.compute_overlap()
    kin = obasis.compute_kinetic()
    na = obasis.compute_nuclear_attraction(mol.coordinates, mol.pseudo_numbers)
    er = obasis.compute_electron_repulsion()
    
    orb_alpha = horton.Orbitals(obasis.nbasis)
    orb_beta = horton.Orbitals(obasis.nbasis)
    horton.guess_core_hamiltonian(olp, kin+na, orb_alpha, orb_beta)
    
    external = {'nn': horton.compute_nucnuc(mol.coordinates, mol.pseudo_numbers)}
    
    terms = [horton.UTwoIndexTerm(kin, 'kin'), horton.UDirectTerm(er, 'hartree'), horton.UExchangeTerm(er, 'x_hf'), horton.UTwoIndexTerm(na, 'ne')]
    ham = horton.UEffHam(terms, external)
    occ_model = horton.AufbauOccModel(21, 21)
    occ_model.assign(orb_alpha, orb_beta)
    dm_alpha = orb_alpha.to_dm()
    dm_beta = orb_beta.to_dm()
    scf_solver = horton.EDIIS2SCFSolver(1e-5, maxiter=400)
    scf_solver(ham, olp, occ_model, dm_alpha, dm_beta)
    
    ham.reset(dm_alpha, dm_beta)
    energy = ham.compute_energy()

    # integration grid
    rho_alpha = obasis.compute_grid_density_dm(dm_alpha, grid.points)
    rho_beta = obasis.compute_grid_density_dm(dm_beta, grid.points)
    rho_full = rho_alpha + rho_beta
    
    return mol.coordinates, grid, {'mixing': mixing, 'energy': energy, 'density': rho_full, 'Enn': external['nn'], 'label': label}

In [15]:


results = []

# A
baseline, target = [7., 7., 7., 7., 7., 7, 0., 0., 0., 0., 0., 0.], [5., 7., 5., 7., 5., 7., 1., 1., 1., 1., 1., 1.]
#build_base(baseline, target, 0.5, basisset)

# B
baseline, target = [7., 7., 7., 7., 7., 7, 0., 0., 0., 0., 0., 0.], [6., 6., 6., 6., 6., 6., 1., 1., 1., 1., 1., 1.]

# C
baseline, target = [6., 6., 6., 6., 6., 6., 1., 1., 1., 1., 1., 1.], [5., 7., 5., 7., 5., 7., 1., 1., 1., 1., 1., 1.]

MemoryError: 

In [17]:
basisset = 'STO-6G'

coordcache = {}
gridcache = {}
results = []
def _do_run(baseline, target, label):
    for mixingval in np.linspace(0, 1, 11):
        try:
            #{'bond': bond_length, 'mixing': mixing_value, 'energy': ener, 'density': den, 'Enn': nuc, 'reference': '-'.join(map(str, reference))}
            molcoords, g, d = get_hf_density(baseline, target, mixingval, basisset, label)
            results.append(d)
        except:
            print (label, mixingval)
            raise
    coordcache[label] = molcoords
    gridcache[label] = g

_do_run([7., 7., 7., 7., 7., 7, 0., 0., 0., 0., 0., 0.], [5., 7., 5., 7., 5., 7., 1., 1., 1., 1., 1., 1.], 'A')
_do_run([7., 7., 7., 7., 7., 7, 0., 0., 0., 0., 0., 0.], [6., 6., 6., 6., 6., 6., 1., 1., 1., 1., 1., 1.], 'B')
_do_run([6., 6., 6., 6., 6., 6., 1., 1., 1., 1., 1., 1.], [5., 7., 5., 7., 5., 7., 1., 1., 1., 1., 1., 1.], 'C')
    
df = pd.DataFrame(results)
cache = df.sort_values('mixing')

In [18]:
#df.to_pickle('abc.pkl.gz')

In [28]:
def integrate():
    ret = []
    for name, group in cache.groupby('label'.split()):
        rhotilde = np.trapz(group.density.values, group.mixing.values, axis=0)
        #grid = grids[grids.bond == group.bond.unique()[0]].iloc[0]
        grid = gridcache[name]
        coordinates = coordcache[name]
        if name == 'A':
            dZ = np.array([5., 7., 5., 7., 5., 7., 1., 1., 1., 1., 1., 1.]) - np.array([7., 7., 7., 7., 7., 7, 0., 0., 0., 0., 0., 0.])
        if name == 'B':
            dZ = np.array([6., 6., 6., 6., 6., 6., 1., 1., 1., 1., 1., 1.]) - np.array([7., 7., 7., 7., 7., 7, 0., 0., 0., 0., 0., 0.])
        if name == 'C':
            dZ = np.array([5., 7., 5., 7., 5., 7., 1., 1., 1., 1., 1., 1.]) - np.array([6., 6., 6., 6., 6., 6., 1., 1., 1., 1., 1., 1.])
        for atom in range(12):
            contribution = -dZ[atom]*np.sum(rhotilde * grid.weights / np.linalg.norm(grid.points - coordinates[atom], axis=1))
            ret.append({'site': atom, 'energy': contribution, 'path': name})
    return pd.DataFrame(ret)
integrated = integrate()

In [29]:
integrated

,energy,path,site
0,49.384163,A,0
1,-0.000000,A,1
2,49.384182,A,2
3,-0.000000,A,3
4,49.384162,A,4
5,-0.000000,A,5
6,-10.294052,A,6
7,-10.178630,A,7
8,-10.294086,A,8
9,-10.178622,A,9


In [30]:
23.238718+26.152691

49.391408999999996

In [33]:
-26.168458+26.152699

-0.015759000000002743